---------------------------------------------------------------------------------------------------------------------------
This tutorial relies on the example model provided by `dinsar`, which is built from real public datasets belonging to a small area near the Doñana National Park (southern Spain). The model can be accessed directly through  `dinsar.example.get_model()` and the relative paths to the data, through `dinsar.example.get_path()`.
For more information about the data sources, see the file *Data Sources.txt*, located in the same folder as this tutorial.

Fore **more information** about the use of any function, method, class o moduel, use `help` bult-in (or `?`) to acces its documentation:

`>>> help(dinsar.parts)` or press the **tabulator** key in Jupyter Notebook to access an object's methods.

---------------------------------------------------------------------------------------------------------------------------

## 1. Creation of a Model
The dinsar package design holds a data **model** as the main object around which all of the program functionalities spin around. This model will include all the DInSAR datasets and databases that are consulted in the study area and is created through the `Model` class. Since the simplest way of analyzing DInSAR deformation points (also called persistent or permanent scatterers; PS) is through their **aggregation into polygons**, the initialization of this class requieres a polygonal spatial file, that should be introduced as a relative path to its ubication. This way, each of these polygons will define the different analysis areas of the model -analogous to [ADA's](https://open.igme.es/xmlui/bitstream/handle/20.500.12468/708/fast_detection_ground_2017.pdf?sequence=1) (Active Deformation Areas), called in the code as **agregados** and hereinafter as *aggregates*.

Althoug the correct generation of this aggregates requiere the application of specific point aggregation techniques, a simple way to generate this layer of polygons is through the 'Aggregate Points' tool of QGIS.

In [19]:
import dinsar

fname = dinsar.example.get_path('agregados')
Doñana = dinsar.Model(fname, epsg=25830)

You can specify the CRS (**Coordinate Reference System**) number of the layer through the `epsg` argument of the class. That CRS will be considered as the project one, and any layer with a different CRS that is added to the model will be reprojected to that.

It's possile to see the current project CRS through `crs` method. Also, the project's CRS can be changed through the `reproject` method, which will reproject all model's layers to the indicated CRS.

In [20]:
Doñana.crs

<Projected CRS: PROJCS["ETRS89_UTM_zone_30N",GEOGCS["ETRS89",DATUM ...>
Name: ETRS89_UTM_zone_30N
Axis Info [cartesian]:
- [east]: Easting (metre)
- [north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: UTM zone 30N
- method: Transverse Mercator
Datum: European Terrestrial Reference System 1989
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

## 2. Definition of Model's parts

Once the Model of the zone has been created, the next operation to be performed is the definition of the **parts** that will compose it. There are four possible types of parts, each one of them being a different object, and being intensively explained in its documentation. Example: see `Dataset?`.

    . Dataset
    . Piezometria
    . Precipitacion
    . DataBase

While the `Dataset` class hosts a DInSAR dataset, the rest of the classes represent databases, which may have associated spatial information. Among the latter, `Piezometria` and `Precipitacion` are specially designed to host piezometric and pluviometric databases, respectively, with data of any other nature being introduced through the `DataBase` class.

As many parts as desired can be defined and added to the model, requiring in the instantialization of each of them the path to the file location, as well as the name with which they will be referred to, the color with which their content will be displayed or other options (see **kwargs of each function).

### DInSAR datasets definition:   

In [21]:
# Defining Ascending Track (called 'Asc') of Sentinel DInSAR dataset.
fname_1 = dinsar.example.get_path('sentinel-asc')
Asc = dinsar.Dataset(fname_1, name='Asc', color='#99F550')

# Defining Descending Track (alias, 'Desc') of Sentinel DInSAR dataset.
fname_2 = dinsar.example.get_path('sentinel-desc')
Desc = dinsar.Dataset(fname_2, name='Desc', color='#FBA608')

###  Piezometry and Rainfall databases

In [22]:
# Defining piezometry database (called 'Piezo_bbdd'), with other arguments ('sep')
# passed in this case to pd.read_file function.
fname = dinsar.example.get_path('piezometria_bd')
piezo = dinsar.Piezometria(fname, name='Piezo_bbdd', sep='\t')

# Adding spatial data to 'Piezo_bbdd'
shp_fname = dinsar.example.get_path('piezometria_shp')
piezo.append_geometry(shp_fname)

#***************************************************************************************************************************
# Defining pluviometric database (called 'P'), with other arguments ('sep')
# passed in this case to pd.read_file function.
fname = dinsar.example.get_path('precipitacion_bd')
precipi = dinsar.Precipitacion(fname, name='P', sep='\t')

# Adding spatial data to 'P'
shp_precipi = dinsar.example.get_path('precipitacion_shp')
precipi.append_geometry(shp_precipi)

Geometría añadida correctamente.
Geometría añadida correctamente.


To add spatial information to a Part (only for none `Dataset` type ones) you can simply do it through `append_geometry` method. Its correct perform requires the sensors to be named with exactly the same name that it binds in the database.

## 3. Adding parts to the Model

Now all the created parts have to be added to the model, operation that is done through the `append` method, a function of `Model` that returns None.

In [26]:
parts = [Asc, Desc, piezo, precipi]

[Doñana.append(i) for i in parts]

[None, None, None, None]

The creation of this **example model** can also be done **quickly** in the following way, which will be used in the rest of the tutorials, as well as in the code examples:

In [8]:
Doñana = dinsar.example.get_model()

Geometría añadida correctamente.
Geometría añadida correctamente.


While the PS that make up the DInSAR datasets added to the model are **cut** to the aggregates extension, the rest of the parts remain unchanged when they are added to the model. Thus, if after adding the parts to the model you want to work with the datasets individually, you can access them simply through the created object or through the `get` method of the model. In the latter case, keep in mind that it will hosts only the PS located within the aggregates.

For the rest of the databases (`Piezometria`, `Precipitacion` and `DataBase`) it is indifferent to do it one way or another. Example:

In [27]:
Asc != Doñana.get('Asc')

False

In [28]:
piezo == Doñana.get('Piezo_bbdd')

True

## Acceder a otras cosa del modelo

## Funcionalidades varias:

In [ ]:
Doñana.study(2).plot() # Plotear las series temporales del agregado 2

In [ ]:
Doñana.study(2).mapa() # Representar espacialmente el agregado 2 y sus PS.

In [ ]:
 # Cojo los cinco primeros PS del Dataset 'Ascending' (dessde fuera del modelo ) y calculo su deformación promedio
ps = Asc.ps[0:5] 
Asc.subset(ps).plot(plot_average=True)

In [ ]:
# Cojo el primer piezómetro de la base de datos de piezometría y ploteo su serie temporal
piezo = bbdd.piezos[0]
bbdd.plot(piezo)

# A través del método 'take' accedo al array con su serie tempral: --> bbdd.take(piezo)

In [ ]:
# Selecciono la estación de Almonte y ploteo su serie temporal según su desviación acumulada
# Puedo ver qué estaciones hay en la base de datos así: --> precipi.estaciones
precipi.plot('Almonte', values='dsv')

In [ ]:
precipi.estaciones